In [1]:
import pandas as pd
import math
import numpy as np
import torch

In [2]:
d = pd.read_csv('./test.csv')
d

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [3]:
# drop unused columns
d = d.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# drop nan
d = d.dropna()

### Type Casting

In [4]:
d['Pclass'] = pd.to_numeric(d['Pclass']) # cast to int

### Optimizations

In [5]:
# optimizing data

#optimize Age
maxAge = max(d['Age'])
d['Age'] = round(d['Age']/maxAge, 2)

# optimize Fare
d['Fare'] = round(np.log10(d['Fare']+1), 2)

# optimize Pclass
d['Pclass_1'] = np.where(d['Pclass']==1, 1, 0)
d['Pclass_2'] = np.where(d['Pclass']==2, 1, 0)

# optimize gender
d['Sex'] = np.where(d['Sex']=='male',1,0)
d.rename(columns={'Sex' : 'Male'}, inplace=True)

# optimize embarked
d['Embark_S'] = np.where(d['Embarked']=='S',1,0)
d['Embark_C'] = np.where(d['Embarked']=='C',1,0)

# add ones
d['Ones'] = 1

d

,Pclass,Male,Age,SibSp,Parch,Fare,Embarked,Pclass_1,Pclass_2,Embark_S,Embark_C,Ones
0,3,1,0.45,0,0,0.95,Q,0,0,0,0,1
1,3,0,0.62,1,0,0.90,S,0,0,1,0,1
2,2,1,0.82,0,0,1.03,Q,0,1,0,0,1
3,3,1,0.36,0,0,0.99,S,0,0,1,0,1
4,3,0,0.29,1,1,1.12,S,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
409,3,0,0.04,1,1,1.17,S,0,0,1,0,1
411,1,0,0.49,1,0,1.96,Q,1,0,0,0,1
412,3,0,0.37,0,0,0.94,S,0,0,1,0,1
414,1,0,0.51,0,0,2.04,C,1,0,0,1,1


In [6]:
final_data = d[['SibSp', 'Parch', 'Age', 'Fare', 'Pclass_1', 'Pclass_2', 'Embark_S', 'Embark_C', 'Male', 'Ones']]

final_data

,SibSp,Parch,Age,Fare,Pclass_1,Pclass_2,Embark_S,Embark_C,Male,Ones
0,0,0,0.45,0.95,0,0,0,0,1,1
1,1,0,0.62,0.90,0,0,1,0,0,1
2,0,0,0.82,1.03,0,1,0,0,1,1
3,0,0,0.36,0.99,0,0,1,0,1,1
4,1,1,0.29,1.12,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
409,1,1,0.04,1.17,0,0,1,0,0,1
411,1,0,0.49,1.96,1,0,0,0,0,1
412,0,0,0.37,0.94,0,0,1,0,0,1
414,0,0,0.51,2.04,1,0,0,1,0,1


In [7]:
# w = np.array([5,2,3,4,5,6,7,8,9,10])
# w = np.array([0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01])
w = np.array([ 0.0067,  0.0503,  0.0333,  0.1744,  0.0832,  0.0513,  0.0469,  0.0628,
        -0.0694,  0.1017])

In [8]:
# finding the loss

# pred = (final_data*w).sum(axis=1)
# loss = (pred - d['Survived']) ** 2
# total_loss = loss.sum()
# data = {
#     'Predictions': pred,
#     'Survived': d['Survived'],
#     'Loss': loss
# }
# df = pd.DataFrame(data)

# print("Total Loss : ",total_loss)
# df

### Using Torch for Backprop

In [9]:
# data
final_data_t = torch.Tensor(final_data.values)       ; #final_data_t.requires_grad = True
# ground_truth = torch.Tensor(d['Survived'].values)

# weights
w_t = torch.Tensor(w)                                ; #w_t.requires_grad = True

In [10]:
for i in range(1):
    # Forward Pass
    predictions = torch.sum((final_data_t * w_t), dim=1) ;#  predictions.requires_grad = True

    print(round(predictions))
    # loss
    # loss = torch.sum((predictions - ground_truth) ** 2)
    # print(i, ' Total Loss: ', loss)
    
    # # backward pass
    # loss.backward()
    
    # # Nudge the gradients
    # with torch.no_grad():
    #     w_t -= 0.0001 * w_t.grad
    
    # # Reset Gradients
    # w_t.grad.zero_()

tensor([0.2130, 0.3329, 0.2905, 0.2638, 0.4106, 0.2613, 0.2786, 0.4569, 0.3329,
        0.3461, 0.4321, 0.5833, 0.4142, 0.5713, 0.4878, 0.2960, 0.2649, 0.3330,
        0.3446, 0.5194, 0.2416, 0.5518, 0.8483, 0.3154, 0.6216, 0.2655, 0.4420,
        0.4086, 0.4179, 0.5022, 0.5067, 0.2635, 0.3306, 0.2681, 0.3647, 0.2629,
        0.4134, 0.5599, 0.2559, 0.4578, 0.6036, 0.4794, 0.4919, 0.3694, 0.5129,
        0.7869, 0.3718, 0.2602, 0.2541, 0.6854, 0.2522, 0.3468, 0.2511, 0.2753,
        0.7200, 0.2754, 0.4691, 0.4483, 0.8881, 0.2763, 0.2542, 0.3269, 0.4456,
        0.6694, 0.5990, 0.5186, 0.3440, 0.2763, 0.3658, 0.6082, 0.4331, 0.2794,
        0.3240, 0.4292, 0.3690, 0.6511, 0.4387, 0.2541, 0.6032, 0.2575, 0.3229,
        0.2609, 0.5786, 0.3986, 0.2545, 0.3994, 0.3304, 0.2073, 0.3391, 0.3754,
        0.5267, 0.2701, 0.6760, 0.3173, 0.4239, 0.5218, 0.4686, 0.4180, 0.5784,
        0.2520, 0.4207, 0.2528, 0.3494, 0.2573, 0.2606, 0.4580, 0.2639, 0.2555,
        0.2672, 0.3424, 0.3260, 0.6983, 